In [9]:
from pathlib import Path

import pandas as pd

In [10]:
dates_to_visualize = [
    "2019-02",
    "2020-02",
    "2021-02",
    "2022-02",
    # "2023_02",
    # "2024_02",
]

In [11]:
available_data_files = list(Path("data/metrics/nel_collector_provider_usage").glob("*.parquet"))

used_data_files = filter(lambda file: file.stem in dates_to_visualize, available_data_files)

In [12]:
result = None
for data_file in used_data_files:
    data = pd.read_parquet(data_file)
    
    break
    
    # if result is None and len(data) != 0:
    #     result = data
    # else:
    #     result = pd.concat([result, data])
        
data

,date,providers,as_primary,share_as_primary,as_secondary,share_as_secondary,among_fallback
0,2019-02,report-uri.com,318.0,85.945946,0.0,0.0,0.0
1,2019-02,3gl.net,21.0,5.675676,0.0,0.0,0.0
2,2019-02,uriports.com,9.0,2.432432,0.0,0.0,0.0
3,2019-02,gvt2.com,5.0,1.351351,5.0,100.0,8.0
4,2019-02,seloc.club,5.0,1.351351,0.0,0.0,0.0
5,2019-02,bingparachute.com,5.0,1.351351,0.0,0.0,0.0
6,2019-02,gie.im,2.0,0.540541,0.0,0.0,0.0
7,2019-02,3genlabs.net,2.0,0.540541,0.0,0.0,0.0
8,2019-02,reenio.cz,1.0,0.270270,0.0,0.0,0.0
9,2019-02,krvtz.net,1.0,0.270270,0.0,0.0,0.0
